<a href="https://colab.research.google.com/github/JHyunjun/AnomalyDetection/blob/main/240824_1DCNN_AE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [6]:
# # TensorFlow 버전 출력
# print("TensorFlow 버전:", tf.__version__)

# # 사용 가능한 GPU 목록 확인
# gpus = tf.config.list_physical_devices('GPU')
# print("사용 가능한 GPU:", gpus)

# if gpus:
#     # GPU가 감지되면 간단한 연산 수행
#     with tf.device('/GPU:0'):
#         a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
#         b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
#         c = tf.matmul(a, b)
#         print("GPU에서 수행된 행렬 곱셈 결과:")
#         print(c)
# else:
#     print("GPU를 찾을 수 없습니다. CPU에서 실행됩니다.")

# # 현재 사용 중인 장치 확인
# print("현재 사용 중인 장치:", tf.test.gpu_device_name())

# # GPU 사용 가능 여부 확인
# print("GPU 사용 가능:", tf.test.is_built_with_cuda())

In [7]:
import pandas as pd
import matplotlib.pyplot as plt

# 데이터 로드
TRAIN_DF_RAW = pd.read_csv("/content/drive/MyDrive/Colab_Notebooks/AutoEncoder/AnomalyDetection_2408_Dayconb/train.csv")
TEST_DF_RAW = pd.read_csv("/content/drive/MyDrive/Colab_Notebooks/AutoEncoder/AnomalyDetection_2408_Dayconb/test.csv")

COLUMNS_IN_TRAIN_DATASET = TRAIN_DF_RAW.columns.drop(["Timestamp", 'anomaly'])

# TIME_STEP 열 추가 (0부터 시작하는 인덱스)
TRAIN_DF_RAW['TIME_STEP'] = range(len(TRAIN_DF_RAW))

# # 그래프 그리기
# plt.figure(figsize=(10, 6))
# plt.scatter(TRAIN_DF_RAW['TIME_STEP'], TRAIN_DF_RAW['anomaly'], alpha=0.5)

# plt.title('Anomaly Detection Over Time Steps', fontsize=20)
# plt.xlabel('TIME_STEP', fontsize=14)
# plt.ylabel('Anomaly', fontsize=14)
# plt.yticks([0, 1])

# plt.tight_layout()
# plt.show()

# 통계 정보 출력
anomaly_count = TRAIN_DF_RAW['anomaly'].sum()
total_count = len(TRAIN_DF_RAW)
print(f"Total data points: {total_count}")
print(f"Number of anomalies: {anomaly_count}")
print(f"Anomaly ratio: {anomaly_count/total_count*100:.2f}%")

Total data points: 496800
Number of anomalies: 0
Anomaly ratio: 0.00%


In [8]:
# 정규화 과정
TRN_MIN = TRAIN_DF_RAW[COLUMNS_IN_TRAIN_DATASET].min()
TRN_MAX = TRAIN_DF_RAW[COLUMNS_IN_TRAIN_DATASET].max()

def normalize(df):
    ndf = df.copy()
    for c in df.columns:
        if TRN_MAX[c] != TRN_MIN[c]:
            ndf[c] = (df[c] - TRN_MIN[c]) / (TRN_MAX[c] - TRN_MIN[c])
    return ndf

TRAIN_DF = normalize(TRAIN_DF_RAW[COLUMNS_IN_TRAIN_DATASET])
TEST_DF = normalize(TEST_DF_RAW[COLUMNS_IN_TRAIN_DATASET])

print("TRAIN_DF.shape : ", TRAIN_DF.shape)
print("TEST_DF.shape : ", TEST_DF.shape )

TRAIN_DF.shape :  (496800, 51)
TEST_DF.shape :  (450000, 51)


In [9]:
def create_windows(data, window_size):
    windows = []
    for i in range(len(data) - window_size + 1):
        windows.append(data[i:i+window_size])
    return np.array(windows)

# TRAIN_DF를 윈도우로 나누기
window_size = 10
train_windows = create_windows(TRAIN_DF.values, window_size)

print("Original TRAIN_DF shape:", TRAIN_DF.shape)
print("After windowing, train_windows shape:", train_windows.shape)

# TEST_DF도 같은 방식으로 윈도우로 나누기
test_windows = create_windows(TEST_DF.values, window_size)

print("\nOriginal TEST_DF shape:", TEST_DF.shape)
print("After windowing, test_windows shape:", test_windows.shape)

Original TRAIN_DF shape: (496800, 51)
After windowing, train_windows shape: (496791, 10, 51)

Original TEST_DF shape: (450000, 51)
After windowing, test_windows shape: (449991, 10, 51)


In [9]:
# 데이터 준비
train_windows = create_windows(TRAIN_DF.values, window_size)
test_windows = create_windows(TEST_DF.values, window_size)

# 데이터 정규화
scaler = StandardScaler()
train_windows_flat = train_windows.reshape(-1, train_windows.shape[-1])
train_windows_scaled = scaler.fit_transform(train_windows_flat).reshape(train_windows.shape)
test_windows_scaled = scaler.transform(test_windows.reshape(-1, test_windows.shape[-1])).reshape(test_windows.shape)

# 학습 데이터와 검증 데이터 분리
train_data, val_data = train_test_split(train_windows_scaled, test_size=0.2, random_state=42)

# 1D-CNN Autoencoder 모델 정의 (수정됨)
def create_model(input_shape):
    model = models.Sequential([
        layers.Input(shape=input_shape),
        layers.Conv1D(32, 3, activation='relu', padding='same'),
        layers.MaxPooling1D(2, padding='same'),
        layers.Conv1D(16, 3, activation='relu', padding='same'),
        layers.Conv1D(8, 3, activation='relu', padding='same'),
        layers.Conv1D(16, 3, activation='relu', padding='same'),
        layers.UpSampling1D(2),
        layers.Conv1D(32, 3, activation='relu', padding='same'),
        layers.Conv1D(input_shape[-1], 3, activation=None, padding='same')
    ])
    return model

# 모델 생성 및 컴파일
model = create_model(train_data.shape[1:])
model.compile(optimizer='adam', loss='mse')

# 모델 요약 출력
model.summary()

# 모델 학습
history = model.fit(
    train_data, train_data,
    epochs=50,
    batch_size=64,
    validation_data=(val_data, val_data),
    shuffle=True
)


In [ ]:
# 재구성 오차 계산
train_pred = model.predict(train_windows_scaled)
train_mse = np.mean(np.square(train_windows_scaled - train_pred), axis=(1,2))

# 이상치 탐지를 위한 임계값 설정 (예: 99 퍼센타일)
threshold = np.percentile(train_mse, 99)

# 테스트 데이터에 대한 예측 및 이상 탐지
test_pred = model.predict(test_windows_scaled)
test_mse = np.mean(np.square(test_windows_scaled - test_pred), axis=(1,2))
anomalies = test_mse > threshold

# 결과 출력
print(f"Detected anomalies: {np.sum(anomalies)}")
print(f"Anomaly ratio: {np.mean(anomalies)*100:.2f}%")

# 시각화
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(test_mse, label='Reconstruction Error')
plt.axhline(y=threshold, color='r', linestyle='--', label='Threshold')
plt.title('Anomaly Detection Results')
plt.xlabel('Sample')
plt.ylabel('Reconstruction Error')
plt.legend()
plt.show()